# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import azureml
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.webservice import AciWebservice

import requests
import json

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

In [ ]:
ws = Workspace.from_config()

# Choosing a name for experiment
experiment_name = 'automl-exp'

experiment=Experiment(ws, experiment_name)

## Compute Cluster

In [ ]:
cpu_cluster_name = "cpu-cluster"

# Verifying that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',
                                                          max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

The dataset "Heart Failure Prediction" contains 12 clinical features of 299 patients with heart failure and a target variable "DEATH EVENT" indicating if the patient deceased during the follow-up period (bool). Machine Learning can help detect and manage high-risk patients at an earlier stage.

To solve this binary classification problem, 12 features are used to predict possible death events with the help of an ML algorithm.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
key = "Heart Failure Prediction"
description_text = "Heart Failure Prediction DataSet for Udacity Project 3"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 
        print("The dataset is loaded")

if not found:
        # Create Dataset and register it into Workspace
        example_data = "https://raw.githubusercontent.com/ruthussanketh/nd00333-capstone/master/heart_failure_clinical_records_dataset.csv"
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        # Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)

In [ ]:
df = dataset.to_pandas_dataframe()
df.describe()

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

Automl settings: 
1. experiment_timeout_minutes - Maximum amount of time the experiment can take. 20 minutes were set to save time. 
2. max_concurrent_iterations - Maximum number of parallel runs executed on a AmlCompute cluster. Because it should be less than or equal to the number of nodes (5) set when creating the compute cluster, it is set to 5. 
3. The primary metric is Under the Curve Weighted, **AUC_weighted**, to deal with class imbalance. 
4. AutoMLConfig: This is a binary classification task. 
5. The label inside the dataset which it to be predicted is "DEATH_EVENT". 
6. To save time and resources, the enable_early_stopping parameter is set to True.

In [ ]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=cpu_cluster,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [ ]:
# TODO: Submit your experiment
automl_run = experiment.submit(automl_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output = True)
assert(automl_run.get_status() == "Completed")

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
# Retrieving the best automl model
automl_best_run, fitted_automl_model = automl_run.get_output() 

run_dict = automl_best_run.get_details()

print("Best Run ID: ", run_dict["runId"])
print("---"*10)
print("AutoML best run AUC_weighted: ", automl_best_run.get_metrics(name="AUC_weighted"))
print("---"*10)
print("Metrics: ", automl_best_run.get_metrics())
print("---"*10)
print("Fitted model and its hyperparameters : ", fitted_automl_model.steps[-1])

In [ ]:
#TODO: Save the best model
os.makedirs('outputs', exist_ok=True)

model = automl_best_run.register_model(model_name = 'automl_best_model', model_path = './outputs/model.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
automl_best_run.get_file_names()

# Downloading the scoring file 
automl_best_run.download_file('outputs/scoring_file_v_1_0_0.py' , 'score.py')

# Downloading conda environment 
automl_best_run.download_file('outputs/conda_env_v_1_0_0.yml' , 'env.yml')

In [ ]:
from azureml.core import Model
# Creating the environment
env = Environment.get(workspace=ws, name="AzureML-AutoML")
# Creating an inference config
inference_config = InferenceConfig(entry_script="score.py",
                                   environment=env)

# Deploying the model as a web service to an Azure Container Instance (ACI)
service_name = 'hfprediction'
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

webservice = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config)

webservice.wait_for_deployment(show_output=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
scoring_uri = "http://c27402a6-6dd0-4ce8-a9f4-ba6da94647c6.southcentralus.azurecontainer.io/score"

df = df.drop(columns=["DEATH_EVENT"])

input_data = json.dumps({
    'data': df[0:2].to_dict(orient='records')
})


print(input_data)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
# Retrieve logs for this Webservice
webservice.get_logs()

In [ ]:
# Delete service
webservice.delete()